In [52]:
import pandas as pd
import geopandas as gpd

In [53]:
# Vacant Units from Measure M / Rent Board
# Rent Board Data Is Seldom Cleaned or presentable so this is a cleaning phase
vacant_df = pd.read_excel("/Users/darrellowens/Downloads/EmptyHomes_6=25-2025 (1) (1).xlsx")
vacant_df.rename(columns={'Parcel Number' : 'APN'}, inplace=True)
vacant_df['APN'] = vacant_df['APN'].str.replace(" ", "")
# Remove invalid APNs (theres 1, a duplicate)
vacant_df = vacant_df[~vacant_df['APN'].isna()]
# Convert APN strings to ints
# vacant_df['APN'] = vacant_df['APN'].astype(int)
# Remove duplicated entries but only if addresses match
print(f"Prior Vacant Property list: {len(vacant_df)}")
vacant_df = vacant_df.drop_duplicates(subset=['APN', 'Location Address (Master Address)'])
print(f"New  Vacant Property list: {len(vacant_df)}")
# Extract duplicate parcel IDs with different listed addresses.
# These will be reserved for geocoder search
bad_vacant_parcels = vacant_df[vacant_df['APN'].duplicated( keep=False)]
print(f"Bad Parcels with conflicting addrs: {len(bad_vacant_parcels)}")
good_vacant_parcels = vacant_df[~vacant_df['APN'].isin(bad_vacant_parcels ['APN'].unique())]
if (len(good_vacant_parcels) + len(bad_vacant_parcels)) != (len(vacant_df)):
    print("You screwed up.")
    
# Berkeley parcels 
parcels = gpd.read_file("../berkeley_parcels/berkeley_parcels.shp").to_crs("EPSG:4326")
parcels.drop(columns=['APN'], inplace=True)
parcels.rename(columns={'PARCELID' : 'APN'}, inplace=True)
parcels['APN'] = parcels['APN'].astype(str).str.replace(" ", "")

# Prepare to Merge the dataframes
found = good_vacant_parcels[good_vacant_parcels['APN'].isin(parcels['APN'].unique())]
not_found = good_vacant_parcels[~good_vacant_parcels['APN'].isin(parcels['APN'].unique())]
print(len(found))
print(len(not_found))

# Find anomalies using a geocoder
print(not_found['Location Address (Master Address)'].unique())
print(bad_vacant_parcels['Location Address (Master Address)'].unique())
not_found = pd.concat([not_found, bad_vacant_parcels], ignore_index=True)
missing = [
'2321 BLAKE ST BERKELEY CA 94704' ,
'1341 PARKER ST BERKELEY CA 94702 ',
'1412 SPRING WAY BERKELEY CA 94708',
'1727 EUCLID AVE BERKELEY CA 94709' ,
'2336 PIEDMONT AVE BERKELEY CA 94704',
'2012 DWIGHT WAY BERKELEY CA 94704' ,
'2020 DWIGHT WAY BERKELEY CA 94704',
'2001 BLAKE ST BERKELEY CA 94704'
]


Prior Vacant Property list: 382
New  Vacant Property list: 381
Bad Parcels with conflicting addrs: 5
373
3
['2321 BLAKE ST BERKELEY CA 94704' '1341 PARKER ST BERKELEY CA 94702 '
 '1412 SPRING WAY BERKELEY CA 94708']
['1727 EUCLID AVE BERKELEY CA 94709' '2336 PIEDMONT AVE BERKELEY CA 94704'
 '2012 DWIGHT WAY BERKELEY CA 94704' '2020 DWIGHT WAY BERKELEY CA 94704'
 '2001 BLAKE ST BERKELEY CA 94704']


In [54]:
from shapely.geometry import Point
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Geocode the address
geocoded_apns = []
geolocator = Nominatim(user_agent="geo_intersect_app")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=5)
for address in missing:
    location = geocode(address)
    if location:
        print(f"Geocoded Address: {location.address}")
        print(f"Coordinates: ({location.latitude}, {location.longitude})")
        
        # Create a Shapely Point from geocoded location
        point = Point(location.longitude, location.latitude)
        
        # Check if this point intersects any parcel polygons
        intersecting_parcels = parcels[parcels.geometry.contains(point)]
        
        if not intersecting_parcels.empty:
            print("Parcel Found ! APN:")
            print(intersecting_parcels['APN'].iloc[0])
            not_found.loc[not_found['Location Address (Master Address)'] == address, "APN"] = intersecting_parcels['APN'].iloc[0]
        else:
            print("No intersection found.")
    else:
        print("Address could not be geocoded.")
# I believe 2007, Blake Street, LeConte is 2001 Blake St, either a weird address number or parcel number. Its been renamed


Geocoded Address: 2321;2323, Blake Street, LeConte, Berkeley, Alameda County, California, 94701, United States
Coordinates: (37.8639707, -122.2618503)
Parcel Found ! APN:
055183101100
Geocoded Address: 1341;1343, Parker Street, San Pablo Park, Berkeley, Alameda County, California, 94703, United States
Coordinates: (37.8604634, -122.2833947)
Parcel Found ! APN:
054179003200
Geocoded Address: 1412, Spring Way, Gourmet Ghetto, Berkeley, Alameda County, California, 94709, United States
Coordinates: (37.8823425, -122.2639651)
Parcel Found ! APN:
059225305200
Geocoded Address: Yun Lin Temple, 1727, Euclid Avenue, Northside, Berkeley, Alameda County, California, 94709, United States
Coordinates: (37.8773559, -122.2603234)
Parcel Found ! APN:
058219801000
Geocoded Address: Casa Joaquin Murieta, 2336;2338, Piedmont Avenue, Southside, Berkeley, Alameda County, California, 94720, United States
Coordinates: (37.8682475, -122.2524411)
Parcel Found ! APN:
055186900200
Geocoded Address: 2012, Dwight 

In [55]:
not_found.head(10)

,APN,Location Address (Master Address),Owner Name,Owner Mailing Address,"Owner Mailing City, State, Zip",# Units on Parcel,# Vacant Units 2024 Calendar Year,Tax Per Unit,Total Tax Per Parcel,Notes,Flag
0,055183101100,2321 BLAKE ST BERKELEY CA 94704,LIANA TAHIR,131 CHESTER AVE,"SAN FRANCISCO , CA 94132",6,3,6000,18000,NaN,NaN
1,054179003200,1341 PARKER ST BERKELEY CA 94702,XIN & MICHELLE JIN,2343 STUART ST,"BERKELEY, CA 94705",2,2,3000,6000,NaN,NaN
2,059225305200,1412 SPRING WAY BERKELEY CA 94708,RUPERTO JR. & OLGA VISAYA,119 SERRA DRIVE,"VALLEJO, CA 94590",2,2,3000,6000,NaN,NaN
3,058219801000,1727 EUCLID AVE BERKELEY CA 94709,YUN YUN LIN TEMPLE,2959 RUSSELL ST,"BERKELEY, CA 94705-2350 USA",26,26,6000,156000,Religious House. Doesn’t Rent Units Out; They'...,NaN
4,055186900200,2336 PIEDMONT AVE BERKELEY CA 94704,THE GREENLINING INSTITUTE C/O ROSA MARTINEZ,"360 14TH ST., 2ND FL","OAKLAND, CA 94612",22,19,6000,114000,Student Apts in Old House (owned by Greenling ...,Data Error
5,055182203000,2012 DWIGHT WAY BERKELEY CA 94704,DWIGHT BLAKE PROPERTY QOZB LLC c/o LACONIA DE...,1981 N BROADWAY STE 350,"WALNUT CREEK, CA 94596-8213",4,3,6000,18000,NaN,NaN
6,055182202700,2020 DWIGHT WAY BERKELEY CA 94704,BLAKE STREET BERKELEY APTS LLC,4601 PARK RD 450,"CHARLOTTE, NC 28209",3,1,3000,3000,NaN,NaN
7,055182203000,2001 BLAKE ST BERKELEY CA 94704,RICHARD NAGLER,2019 BLAKE ST,"BERKELEY, CA 94704",2,1,3000,3000,"Data Error meant to be 2001, not 2007. 2-units...",Destroyed


In [56]:
# Merge Found APNS
merged_found = found.merge(parcels[['APN', 'Units', 'geometry']], on='APN', how='left')
merged_not_found = not_found.merge(parcels[['APN', 'Units', 'geometry']], on='APN', how='left')
# Final product
vacant_units_w_parcels = pd.concat([merged_not_found, merged_found], ignore_index=True)
vacant_units_w_parcels = gpd.GeoDataFrame(vacant_units_w_parcels, crs='EPSG:4326', geometry='geometry')


In [57]:
# Get Development Pipeline from HCD's Permitting Data of Berkeley
# Downloading HCD APR Data into a dataframe called df_a2
import requests
import io
url = "https://data.ca.gov/dataset/81b0841f-2802-403e-b48e-2ef4b751f77c/resource/fe505d9b-8c36-42ba-ba30-08bc4f34e022/download/table-a2-combined.csv"
s = requests.get(url).content
df_a2 = pd.read_csv(io.StringIO(s.decode("utf-8")))

# Pull out all rows between 2018 - 2023
berk_apr = df_a2[df_a2['JURS_NAME'] == "BERKELEY"]
berk_apr["APN"] = berk_apr["APN"].str.replace(" ", "")
# Group the rows by City and sum up their building permits per city
berk_apr.groupby(berk_apr["APN"])[['NO_ENTITLEMENTS', "NO_BUILDING_PERMITS"]].sum()
# Merge into the vacancy 
vacant_units_w_parcels_and_dev = vacant_units_w_parcels.merge(berk_apr[['NO_ENTITLEMENTS',"NO_BUILDING_PERMITS", 'APN' ]], on='APN', how='left')
vacant_units_w_parcels_and_dev['NO_ENTITLEMENTS'] = vacant_units_w_parcels_and_dev['NO_ENTITLEMENTS'].fillna("N/A")
vacant_units_w_parcels_and_dev["NO_BUILDING_PERMITS"] = vacant_units_w_parcels_and_dev["NO_BUILDING_PERMITS"].fillna("N/A")


/var/folders/2k/1029_3wx62vgt24s0p06bzm80000gn/T/ipykernel_79522/197519868.py:7: DtypeWarning: Columns (3,6,17,40,41,42,43,49,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df_a2 = pd.read_csv(io.StringIO(s.decode("utf-8")))
/var/folders/2k/1029_3wx62vgt24s0p06bzm80000gn/T/ipykernel_79522/197519868.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  berk_apr["APN"] = berk_apr["APN"].str.replace(" ", "")


In [59]:
import numpy as np
vacant_units_w_parcels_and_dev['color'] = np.where(
    vacant_units_w_parcels_and_dev['NO_BUILDING_PERMITS'] != 'N/A',
    'red',
    'blue'
)
vacant_units_w_parcels_and_dev['color'] = np.where( 
    vacant_units_w_parcels_and_dev['Flag'] == "Fire", 
    'orange',
    vacant_units_w_parcels_and_dev['color'] 
)
vacant_units_w_parcels_and_dev['color'] = np.where( 
    vacant_units_w_parcels_and_dev['Flag'] == "Destroyed", 
    'black',
    vacant_units_w_parcels_and_dev['color'] 
)
vacant_units_w_parcels_and_dev['color'] = np.where( 
    vacant_units_w_parcels_and_dev['Flag'] =="Longtimer", 
    'green',
    vacant_units_w_parcels_and_dev['color'] 
)
vacant_units_w_parcels_and_dev['color'] = np.where( 
    vacant_units_w_parcels_and_dev['Flag'] == "Data Error", 
    'pink',
    vacant_units_w_parcels_and_dev['color'] 
)
# Properties I know are Demolished
# Plot using .explore() with color mapping
vacant_units_w_parcels_and_dev.explore(color=vacant_units_w_parcels_and_dev['color'])

In [72]:
# Number of vacancies by Council District 
council_dist = gpd.read_file("/Users/darrellowens/Documents/Census Housing Stats/berkeley_parcels/berkeley_council_districts.shp").to_crs("EPSG:4326")
vacant_units_w_parcels_and_dev['centroid'] = vacant_units_w_parcels_and_dev.geometry.centroid
vacant_units_w_parcels_and_dev_council = gpd.sjoin(vacant_units_w_parcels_and_dev.set_geometry('centroid'), council_dist[['DISTRICT', 'geometry']], how='left', predicate='intersects')

# Aggregate Vacant and Total Units by District (Total Units will derive from Rent Registry)
parcels['centroid']= parcels.geometry.centroid
parcels_and_council = gpd.sjoin(parcels.set_geometry('centroid'), council_dist[['DISTRICT', 'geometry']], how='left', predicate='intersects')


vacant_units_by_district = vacant_units_w_parcels_and_dev_council.groupby("DISTRICT")["# Vacant Units 2024 Calendar Year"].sum()
total_units_by_district = parcels_and_council.groupby("DISTRICT")["Units"].sum()

# Remove the Error Parcels for Comparison
vacant_units_w_parcels_and_dev_council_no_error = vacant_units_w_parcels_and_dev_council[vacant_units_w_parcels_and_dev_council['Flag'] != 'Data Error']
vacant_units_by_district_no_error = vacant_units_w_parcels_and_dev_council_no_error.groupby("DISTRICT")["# Vacant Units 2024 Calendar Year"].sum()


# Convert into Dataframe
summary_df = pd.DataFrame({
    'VACANT UNITS': vacant_units_by_district,
    'VACANT UNITS w/o Data ERROR' : vacant_units_by_district_no_error,
    'TOTAL UNITS': total_units_by_district
})

# Calculate the percentage vacant
summary_df['PERCENT VACANT OF TOTAL UNITS'] = ((summary_df['VACANT UNITS'] / summary_df['TOTAL UNITS']) * 100).round(2)
summary_df['PERCENT VACANT no Error OF TOTAL UNITS'] = ((summary_df['VACANT UNITS w/o Data ERROR'] / summary_df['TOTAL UNITS']) * 100).round(2)

/var/folders/2k/1029_3wx62vgt24s0p06bzm80000gn/T/ipykernel_79522/3347442059.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  vacant_units_w_parcels_and_dev['centroid'] = vacant_units_w_parcels_and_dev.geometry.centroid
/var/folders/2k/1029_3wx62vgt24s0p06bzm80000gn/T/ipykernel_79522/3347442059.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  parcels['centroid']= parcels.geometry.centroid


In [73]:
summary_df

,VACANT UNITS,VACANT UNITS w/o Data ERROR,TOTAL UNITS,PERCENT VACANT OF TOTAL UNITS,PERCENT VACANT no Error OF TOTAL UNITS
DISTRICT,,,,,
1,104,104,15057.0,0.69,0.69
2,90,90,13227.0,0.68,0.68
3,118,118,11798.0,1.00,1.00
4,200,192,17182.0,1.16,1.12
5,41,41,7334.0,0.56,0.56
6,91,91,6925.0,1.31,1.31
7,178,75,3646.0,4.88,2.06
8,93,85,8932.0,1.04,0.95
